# <img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# Convert ALOS2 Data to GCOV

In preparation for NISAR, this notebook converts ALOS-2 SLC data to NISAR RSLC to NISAR GCOV.


These steps can be run in the command line, details are below.

In [ ]:
import os
import sys
from pathlib import Path
import subprocess
import fnmatch
import zipfile
import h5py
from string import Template
from osgeo import gdal
import numpy as np
import math
import boto3
import shutil

## Choose your place to save your outputs. 

The home directory should be used (not scratch) unless very large storage space is needed. Files will be moved to the S3 bucket after processing. 

In [ ]:
notebook_dir = Path(os.getcwd())
working_dir = Path('/scratch/alex_eco_test/')


In [ ]:
bucket_name = "nisar-st-data-ondemand"
s3_path = 'alex/ALOS2_zip/'
s3 = boto3.client("s3")
response = s3.list_objects_v2(
            Bucket=bucket_name,
            Prefix = s3_path)
            # MaxKeys=100)
            # Delimiter = '/')
contents = response.get('Contents')
existing_ALOS2_zipfiles = [contents[i].get('Key').split('/')[-1] for i in range(len(contents)) if '.zip' in contents[i].get('Key')]
existing_ALOS2_zippaths = [contents[i].get('Key') for i in range(len(contents)) if '.zip' in contents[i].get('Key')]


## Choose your AOI

New directories will be made to store files for this AOI

In [ ]:
aoi = 'little_river'

aoi_dir = working_dir/aoi
RSLC_dir = aoi_dir / 'RSLC'
ALOS2_dir = aoi_dir / 'ALOS2'
DEM_dir = aoi_dir / 'DEM'
GCOV_dir = aoi_dir / 'GCOV'
TMP_dir = aoi_dir / 'TMP'

Path(aoi_dir).mkdir(parents=True, exist_ok=True)
Path(RSLC_dir).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir).mkdir(parents=True, exist_ok=True)
Path(DEM_dir).mkdir(parents=True, exist_ok=True)
Path(GCOV_dir).mkdir(parents=True, exist_ok=True)
Path(TMP_dir).mkdir(parents=True, exist_ok=True)



## Copy the ALOS2 URLs into a list

In [ ]:
## Example using Little River, FP6-7
list_of_ALOS_SLCs = [
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000436289_001001_ALOS2387550620-210727.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000436282_001001_ALOS2393760620-210907.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000436278_001001_ALOS2395830620-210921.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000436276_001001_ALOS2397900620-211005.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000436274_001001_ALOS2399970620-211019.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000436293_001001_ALOS2402040620-211102.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000436327_001001_ALOS2404110620-211116.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000437322_001001_ALOS2404260840-211117.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000438368_001001_ALOS2406180620-211130.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000462472_001001_ALOS2414460620-220125.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000463999_001001_ALOS2416530620-220208.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000463996_001001_ALOS2418600620-220222.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000464635_001001_ALOS2420670620-220308.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000464630_001001_ALOS2422740620-220322.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000469345_001001_ALOS2424810620-220405.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000471233_001001_ALOS2431020620-220517.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000473052_001001_ALOS2437230620-220628.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000474588_001001_ALOS2439300620-220712.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000474967_001001_ALOS2441370620-220726.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000476738_001001_ALOS2443440620-220809.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000478836_001001_ALOS2445510620-220823.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000480756_001001_ALOS2447580620-220906.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000482968_001001_ALOS2449650620-220920.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000484610_001001_ALOS2451720620-221004.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000485981_001001_ALOS2453790620-221018.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000487640_001001_ALOS2455860620-221101.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000489256_001001_ALOS2457930620-221115.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000493225_001001_ALOS2460000620-221129.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000495194_001001_ALOS2462070620-221213.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000496585_001001_ALOS2464140620-221227.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000499758_001001_ALOS2466210620-230110.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000501405_001001_ALOS2468280620-230124.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000503163_001001_ALOS2470350620-230207.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000503979_001001_ALOS2472420620-230221.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000504996_001001_ALOS2474490620-230307.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000506845_001001_ALOS2476560620-230321.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000507911_001001_ALOS2478630620-230404.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000509958_001001_ALOS2480700620-230418.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000511821_001001_ALOS2482770620-230502.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000514247_001001_ALOS2484840620-230516.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000515881_001001_ALOS2486910620-230530.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000517708_001001_ALOS2488980620-230613.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000519139_001001_ALOS2491050620-230627.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000521175_001001_ALOS2493120620-230711.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000523789_001001_ALOS2495190620-230725.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000526013_001001_ALOS2497260620-230808.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000527821_001001_ALOS2499330620-230822.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000529048_001001_ALOS2501400620-230905.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000530759_001001_ALOS2503470620-230919.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000532669_001001_ALOS2505540620-231003.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000534919_001001_ALOS2507610620-231017.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000536906_001001_ALOS2509680620-231031.zip',
                    'https://cumulus.asf.alaska.edu/L1.1/A4/0000538007_001001_ALOS2511750620-231114.zip',
                    ]

## Download the ALOS2 files using wget

In [ ]:
ALOS2_ids = []
response = s3.list_objects_v2(
            Bucket=bucket_name,
            Prefix = s3_path)
            # MaxKeys=100)
            # Delimiter = '/')
contents = response.get('Contents')
existing_ALOS2_zipfiles = [contents[i].get('Key').split('/')[-1] for i in range(len(contents)) if '.zip' in contents[i].get('Key')]
existing_ALOS2_zippaths = [contents[i].get('Key') for i in range(len(contents)) if '.zip' in contents[i].get('Key')]

for file in list_of_ALOS_SLCs[:]:
    filename =  file.split('/')[-1]
    ALOS2_ids.append(filename)
    print('Requested File: ', filename)
    print(os.path.isdir(ALOS2_dir/filename.split('/')[-1][:-4]))
    print(os.path.isfile(ALOS2_dir/filename))
    print((filename in existing_ALOS2_zipfiles))
    if os.path.isdir(ALOS2_dir/filename.split('/')[-1][:-4])& (filename in existing_ALOS2_zipfiles):
        print('ALOS2 is stored on S3 and already available locally and unzipped')
    
    elif os.path.isfile(ALOS2_dir/filename)==False & (filename in existing_ALOS2_zipfiles)==True:
        i = existing_ALOS2_zipfiles.index(filename)
        s3_path_new = existing_ALOS2_zippaths[i]
        print('\tALOS2 zip is already available at S3 PATH: ', s3_path_new)
        s3.download_file(bucket_name, s3_path_new , filename)
        print('\tMove ALOS2 zip from S3 to local')
        shutil.move(notebook_dir/filename, ALOS2_dir/filename)
        
    elif os.path.isfile(ALOS2_dir/filename)==False & (filename in existing_ALOS2_zipfiles)==False:
        print('\tALOS2 is not available anywhere')
        print('\tDownloading ALOS2 zip ')
        command = f"wget -P {ALOS2_dir} -q {file}"
        output = subprocess.check_output(command, shell=True)
        print('\tMoving a copy ALOS2 zip to S3 bucket')
        s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s' %(s3_path,filename))
    
    elif os.path.isfile(ALOS2_dir/filename)==True & (filename in existing_ALOS2_zipfiles)==False:
        print('\tALOS2 is available locally, but not on S3')
        print('\tMoving a copy ALOS2 zip to S3 bucket')
        s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s' %(s3_path,filename))

    else: 
        print('\tALOS2 zip file exists locally and on S3')
        
    if os.path.isdir(ALOS2_dir/filename.split('/')[-1][:-4])==False:
        print('\tunzipping the ALOS2 file locally')
        try:
            with zipfile.ZipFile(ALOS2_dir/filename, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/filename.split('/')[-1][:-4])
        except:
            print('\tALOS2 zip is bad, re-downloading')
            os.remove(ALOS2_dir/filename)
            command = f"wget -P {ALOS2_dir} -q {file}"
            output = subprocess.check_output(command, shell=True)
            print('\t\tMoving a copy ALOS2 zip to S3 bucket')
            s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s' %(s3_path,filename))
            with zipfile.ZipFile(ALOS2_dir/filename, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/filename.split('/')[-1][:-4])
        print('\t\tdeleting the local zip file')
        # os.remove(ALOS2_dir/filename) ## comment to remove the alos2 zip file from your local workspace
    

## Get a list of folders for each ALOS2 file

In [ ]:
ALOS2folders = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(dirnames,'*')]
ALOS2folders.sort(reverse=True)
ALOS2folders

## Convert ALOS2 SLC to NISAR RSLC format

In [ ]:
for ALOS2folder in ALOS2folders[:]:
    ALOS2_id = ALOS2folder.split('/')[-1]
    print('')
    print(ALOS2_id)
    
    ##ALOS2 SLC --> NISAR RSLC
    command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {RSLC_dir/ALOS2_id}.h5"
  
    if os.path.isfile(f"{RSLC_dir/ALOS2_id}.h5")==True:
        print('NISAR RSLC already converted')
    else:
            
        try: os.remove(f"{RSLC_dir/ALOS2_id}.h5")
        except:''
        output = subprocess.check_output(command, shell=True)    
        print('')
        print(command)  
        

In [ ]:
ALOS2_RSLCs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(RSLC_dir)
                for f in fnmatch.filter(files,'*')]
ALOS2_RSLCs.sort(reverse=True)
ALOS2_RSLCs



## Get a DEM for the first ALOS2 file in the stack 

In [ ]:

ALOS2_ref = ALOS2_RSLCs[0].split('/')[-1][:-3]
print('')
print(ALOS2_ref)

## Get NISAR DEM
command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p {RSLC_dir/ALOS2_ref}.h5 -o {DEM_dir/ALOS2_ref}.vrt"
   

if os.path.isfile(f"{DEM_dir/ALOS2_ref}.vrt")==False:
    print('')
    print(command) 
    output = subprocess.check_output(command, shell=True)

## Process NISAR RSLC to GCOV

### If you want to change to UTM coordiates, set utm = True and choose the x and y posting. The default is 20m

In [ ]:
x_posting = ''
y_posting = ''
EPSG = ''

utm = True
if utm:
    src = gdal.Open(f"{DEM_dir/ALOS2_ref}.vrt")
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)

    x1,y1,x2,y2 = math.floor(ulx),math.floor(uly),math.floor(lrx),math.floor(lry)
    zone = int(np.ceil((ulx + 180)/6))

    if y1>=0:
        EPSG = 32600+zone
    elif y1<0:
        EPSG = 32600+zone

    x_posting = 20
    y_posting = 20
    
print(EPSG)



## Run the GCOV processor on the first/reference image in the stack. 
This image will be used to determine the bounding box of the remaining images in the stack

In [ ]:
filein = open('gcov_template.yaml')
template = Template(filein.read())
replacements = {'inputfile':  f"{RSLC_dir/ALOS2_ref}.h5",
                'outputfile':  f"{GCOV_dir/ALOS2_ref}_gcov_%s.h5" %(EPSG),
                'demfile': f"{DEM_dir/ALOS2_ref}.vrt",
                'tmp': f"{TMP_dir}/",
                'epsg': EPSG,
                'xposting': x_posting,
                'yposting': y_posting,
                'top_left_y': '',
                'top_left_x' : '',
                'bottom_right_y':'',
                'bottom_right_x':'',
                'y_snap':'',
                'x_snap':''
                }
makeoutput = template.substitute(replacements)
file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_ref),'w')
file.write(makeoutput)
file.close()
filein.close()

command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_ref}.yaml"

if os.path.isfile(f"{GCOV_dir/ALOS2_ref}_gcov_utm.h5")==False:
    print('')
    print(command)
    output = subprocess.check_output(command, shell=True)


## Get top left coordinates of bottom right coordinates of reference image

In [ ]:
f = h5py.File(f"{GCOV_dir/ALOS2_ref}_gcov_%s.h5" %(EPSG), "r") 
a_group_key = list(f.keys())[0]
ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object

ulx = x_posting * round(ds_x[0]/x_posting)
lrx = x_posting * round(ds_x[-1]/x_posting)
uly = y_posting * round(ds_y[0]/y_posting)
lry = y_posting * round(ds_y[-1]/y_posting)

print('Force top left to be: %s %s' %(ulx,uly))
print('Force top right to be: %s %s' %(lrx,lry))



## Process the remaining RSLC files to GCOV using the bounding box set by the reference images

In [ ]:
for ALOS2_RSLC in ALOS2_RSLCs[:]:
    ALOS2_id = ALOS2_RSLC.split('/')[-1][:-3]
    print('')
    print(ALOS2_id)

    filein = open('gcov_template.yaml')
    template = Template(filein.read())
    replacements = {'inputfile':  f"{RSLC_dir/ALOS2_id}.h5",
                    'outputfile':  f"{GCOV_dir/ALOS2_id}_gcov_%s.h5" %(EPSG),
                    'demfile': f"{DEM_dir/ALOS2_ref}.vrt",
                    'tmp': f"{TMP_dir}/",
                    'epsg': EPSG,
                    'xposting': x_posting,
                    'yposting': y_posting,
                    'top_left_y': uly,
                    'top_left_x':ulx,
                    'bottom_right_y':lry,
                    'bottom_right_x': lrx,
                    'y_snap':'',#y_posting,
                    'x_snap':''#x_posting
                    }
    makeoutput = template.substitute(replacements)
    file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_id),'w')
    file.write(makeoutput)
    file.close()
    filein.close()

    command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_id}.yaml"
    print('')
    print(command)
    if os.path.isfile(f"{GCOV_dir/ALOS2_id}_gcov_utm.h5")==False:
        output = subprocess.check_output(command, shell=True)


In [ ]:
for ALOS2_RSLC in ALOS2_RSLCs[:]:
    ALOS2_id = ALOS2_RSLC.split('/')[-1][:-3]
    print('')
    print(ALOS2_id)
    f = h5py.File(f"{GCOV_dir/ALOS2_id}_gcov_%s.h5" %(EPSG), "r") 
    a_group_key = list(f.keys())[0]
    ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
    ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object

    ulx = x_posting * round(ds_x[0]/x_posting)
    lrx = x_posting * round(ds_x[-1]/x_posting)
    uly = y_posting * round(ds_y[0]/y_posting)
    lry = y_posting * round(ds_y[-1]/y_posting)

    print('Force top left to be: %s %s' %(ulx,uly))
    print('Force top right to be: %s %s' %(lrx,lry))



## See final list of NISAR GCOV data files

In [ ]:
NISAR_GCOVs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(GCOV_dir)
                for f in fnmatch.filter(files,'*.h5')]
NISAR_GCOVs.sort(reverse=True)
NISAR_GCOVs

In [ ]:
## Command line option

######## ALOS2 SLC to NISAR RSLC
# for f in /scratch/alex_eco_test/ALOS2/southfork/*/; 
# do python /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i $f -o "${f%.*}".h5; 
# done

######## Get NISAR DEM
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p $f -o "${f%.*}".vrt; 
# done

######## NISAR RSLC to NISAR GCOV
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*.yaml; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py $f; 
# done